### TextRank Extractive Summarization method

In [1]:
import pandas as pd

# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# from nltk.probability import FreqDist
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize, sent_tokenize
# from collections import defaultdict

In [8]:
# df = pd.read_csv("../cleaned_data/cna_cleaned_with_NER.csv")
# content = df[["url", "cleaned_content"]]

In [9]:
# from summarizer import summarize

# title = "Sports"

# url = []
# s = []

# for index, row in content.iterrows():
#     url.append(row["url"])
#     summary = summarize(title, row['cleaned_content'])
#     s.append(summary)


# df = pd.DataFrame(list(zip(url, s)), columns =['url', 'summarised_content'])

# df.to_csv("cna_extractive_summary.csv")

In [ ]:
# evaluate model =acc etc 
# rouge technique
# find the best technique
# golden truth , put into hugging face, chatgpt 

In [11]:
import pandas as pd
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
from datetime import datetime as dt


### Helper function

def text_to_sent_list(text, 
                      nlp = spacy.load("en_core_web_lg"), 
                      embedder = SentenceTransformer('distilbert-base-nli-mean-tokens'),
                      min_len=2):
    
    ''' Returns cleaned article sentences and BERT sentence embeddings'''
    
    #convert to list of sentences
    text = nlp(text)
    sents = list(text.sents)
    #remove short sentences by threshhold                                                                                                
    sents_clean = [sentence.text for sentence in sents if len(sentence)> min_len]
    #remove entries with empty list
    sents_clean = [sentence for sentence in sents_clean if len(sentence)!=0]
    #embed sentences (deafult uses BERT SentenceTransformer)
    sents_embedding= np.array(embedder.encode(sents_clean, convert_to_tensor=True))
    
    return sents_clean, sents_embedding



### Script

# output_file = 'train_stats_df_processed_extr_5000.pickle'  
#load full extractive df
df = pd.read_csv("../cleaned_data/cna_cleaned_with_NER.csv")
content = df["cleaned_content"]

#truncate for local computation
# df= df.head(5000).reset_index(drop=True)

#load nlp and embedder
nlp = spacy.load("en_core_web_lg")
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

t1 = dt.now()
print(t1)

#extract clean sentence list and sentence embedding for each article TEXT
f = lambda text: text_to_sent_list(text, nlp=nlp, embedder=embedder, min_len=2)
s_interim_tuple = content.apply(f)

content['text_clean'] = s_interim_tuple.apply(lambda x: x[0])
content['text_embedding'] = s_interim_tuple.apply(lambda x: x[1])

#extract clean sentence list and sentence embedding for each article SUMMARY
f = lambda summ: text_to_sent_list(summ, nlp=nlp, embedder=embedder, min_len=0)
s_interim_tuple = content['summary'].apply(f)

content['summary_clean'] = s_interim_tuple.apply(lambda x: x[0])
content['summary_embedding'] = s_interim_tuple.apply(lambda x: x[1])

# with open(output_file, 'wb') as handle:                                     
#     pickle.dump(df, handle)

t2=dt.now()
print(t2)
print(t2-t1)

content.head(10)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

2023-03-30 18:02:02.590613


/var/folders/w6/bkhgxy792vxf30hdj_b9m6200000gn/T/ipykernel_18673/2052824106.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['text_clean'] = s_interim_tuple.apply(lambda x: x[0])


KeyError: 'summary'

In [ ]:
content